In [14]:
import os
import sys
import time
import glob
import numpy as np
import torch
import utils
from PIL import Image
import logging
import argparse
import torch.utils
import torch.backends.cudnn as cudnn
import torch.nn as nn
from torch.autograd import Variable

from model import *
from multi_read_data import MemoryFriendlyLoader


parser = argparse.ArgumentParser("SCI")
parser.add_argument('--batch_size', type=int, default=1, help='batch size')
parser.add_argument('--cuda', default=True, type=bool, help='Use CUDA to train model')
parser.add_argument('--gpu', type=str, default='0', help='gpu device id')
parser.add_argument('--seed', type=int, default=2, help='random seed')
parser.add_argument('--epochs', type=int, default=1000, help='epochs')
parser.add_argument('--lr', type=float, default=0.0003, help='learning rate')
parser.add_argument('--stage', type=int, default=3, help='epochs')
parser.add_argument('--save', type=str, default='EXP/', help='location of the data corpus')

args = parser.parse_args()

os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

args.save = args.save + '/' + 'Train-{}'.format(time.strftime("%Y%m%d-%H%M%S"))
utils.create_exp_dir(args.save, scripts_to_save=glob.glob('*.py'))
model_path = args.save + '/model_epochs/'
os.makedirs(model_path, exist_ok=True)
image_path = args.save + '/image_epochs/'
os.makedirs(image_path, exist_ok=True)

log_format = '%(asctime)s %(message)s'
logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                    format=log_format, datefmt='%m/%d %I:%M:%S %p')
fh = logging.FileHandler(os.path.join(args.save, 'log.txt'))
fh.setFormatter(logging.Formatter(log_format))
logging.getLogger().addHandler(fh)

logging.info("train file name = %s", os.path.split(__file__))

if torch.cuda.is_available():
    if args.cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    if not args.cuda:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')


def save_images(tensor, path):
    image_numpy = tensor[0].cpu().float().numpy()
    image_numpy = (np.transpose(image_numpy, (1, 2, 0)))
    im = Image.fromarray(np.clip(image_numpy * 255.0, 0, 255.0).astype('uint8'))
    im.save(path, 'png')


def main():
    if not torch.cuda.is_available():
        logging.info('no gpu device available')
        sys.exit(1)

    np.random.seed(args.seed)
    cudnn.benchmark = True
    torch.manual_seed(args.seed)
    cudnn.enabled = True
    torch.cuda.manual_seed(args.seed)
    logging.info('gpu device = %s' % args.gpu)
    logging.info("args = %s", args)


    model = Network(stage=args.stage)

    model.enhance.in_conv.apply(model.weights_init)
    model.enhance.conv.apply(model.weights_init)
    model.enhance.out_conv.apply(model.weights_init)
    model.calibrate.in_conv.apply(model.weights_init)
    model.calibrate.convs.apply(model.weights_init)
    model.calibrate.out_conv.apply(model.weights_init)

    model = model.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, betas=(0.9, 0.999), weight_decay=3e-4)
    MB = utils.count_parameters_in_MB(model)
    logging.info("model size = %f", MB)
    print(MB)


    TrainDataset = MemoryFriendlyLoader(img_dir='../LOL/train480/high')
    ValDataset = MemoryFriendlyLoader(img_dir='../LOL/val5/high')

    train_queue = torch.utils.data.DataLoader(
        TrainDataset, batch_size=args.batch_size,
        pin_memory=True, num_workers=0, shuffle=True)
    val_queue = torch.utils.data.DataLoader(
        TestDataset, batch_size=1,
        pin_memory=True, num_workers=0, shuffle=True)


    total_step = 0

    for epoch in range(args.epochs):
        model.train()
        losses = []
        for batch_idx, (in_, sem_, imgname_, semname_ ) in enumerate(train_queue):
            total_step += 1
            input = Variable(in_, requires_grad=False).cuda()

            optimizer.zero_grad()
            loss = model._loss(input)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()

            losses.append(loss.item())
            logging.info('train-epoch %03d %03d %f', epoch, batch_idx, loss)



        logging.info('train-epoch %03d %f', epoch, np.average(losses))
        utils.save(model, os.path.join(model_path, 'weights_%d.pt' % epoch))

        if epoch % 1 == 0 and total_step != 0:
            logging.info('train %03d %f', epoch, loss)
            model.eval()
            with torch.no_grad():
                for batch_idx, (in_, sem_, imgname_, semname_ ) in enumerate(val_queue):
                    in_ = Variable(in_, volatile=True).cuda()
                    image_name = imgname_[0].split('\\')[-1].split('.')[0]
                    illu_list, ref_list, input_list, atten= model(in_, sem_)
                    u_name = '%s.png' % (image_name + '_' + str(epoch))
                    u_path = image_path + '/' + u_name
                    save_images(ref_list[0], u_path)
        break

if __name__ == '__main__':
    main()


usage: SCI [-h] [--batch_size BATCH_SIZE] [--cuda CUDA] [--gpu GPU]
           [--seed SEED] [--epochs EPOCHS] [--lr LR] [--stage STAGE]
           [--save SAVE]
SCI: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-b866225e-9a4c-4b3c-9a2f-733bac51211d.json


SystemExit: 2

In [15]:
%tb

SystemExit: 2

In [4]:
import torch.nn as nn
in_conv = torch.nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(channels),
            nn.ReLU()
        )

In [6]:
isinstance(in_conv, nn.Conv2d)

False

In [24]:
import torch
from model import *

In [ ]:
model = Network(stage=3)

In [25]:
for param in model.enhance.fusion.parameters():
    print(param)

Parameter containing:
tensor([[[[-0.0615,  0.0446, -0.0603],
          [-0.1219, -0.1203,  0.0923],
          [-0.0959, -0.0020,  0.0384]],

         [[-0.0504, -0.1005,  0.0755],
          [ 0.0695, -0.1062,  0.1188],
          [ 0.0069, -0.0851, -0.0077]],

         [[-0.0364,  0.0819, -0.0038],
          [-0.0341,  0.0353, -0.0937],
          [ 0.1181, -0.0729, -0.1076]],

         [[-0.0512,  0.1298, -0.0350],
          [-0.0463, -0.0804, -0.0572],
          [-0.0265,  0.0054, -0.0262]],

         [[ 0.1109, -0.0459,  0.0766],
          [ 0.0192, -0.0739,  0.0225],
          [-0.0828,  0.0846, -0.0388]],

         [[-0.1354,  0.0335, -0.0732],
          [ 0.0974,  0.0605, -0.0247],
          [-0.0799,  0.0098, -0.0854]]],


        [[[ 0.0582, -0.0868,  0.1019],
          [ 0.0236, -0.1215, -0.0096],
          [ 0.0979,  0.0965, -0.0597]],

         [[ 0.1062,  0.1154, -0.1175],
          [-0.1009,  0.0412, -0.0891],
          [ 0.0418,  0.0666, -0.0757]],

         [[-0.0963, -0.0

In [1]:
import torch
from model import *

In [13]:
model = Network(stage=3)
model_dict = model.state_dict()
model_dict

OrderedDict([('enhance.in_conv.0.weight',
              tensor([[[[-0.0304, -0.0355,  0.1075],
                        [-0.0085,  0.0898,  0.0804],
                        [-0.1262,  0.0687, -0.1742]],
              
                       [[-0.0835,  0.1682,  0.1076],
                        [-0.0210,  0.0949,  0.1862],
                        [ 0.1011, -0.0882,  0.0668]],
              
                       [[ 0.0273,  0.1336, -0.0521],
                        [ 0.0319, -0.0811,  0.1521],
                        [ 0.1813, -0.1443, -0.0491]]],
              
              
                      [[[-0.1253,  0.1769,  0.0344],
                        [-0.0332,  0.1822, -0.1248],
                        [-0.1203, -0.1383,  0.0830]],
              
                       [[ 0.0855, -0.1518,  0.0277],
                        [-0.1201,  0.0537, -0.0077],
                        [-0.1427,  0.0816,  0.0941]],
              
                       [[ 0.1259, -0.0746, -0.1254],
              

In [9]:
for k, v in model_dict.items():
    print(k)

enhance.in_conv.0.weight
enhance.in_conv.0.bias
enhance.fusion.conv.0.weight
enhance.fusion.conv.0.bias
enhance.fusion.conv.1.weight
enhance.fusion.conv.1.bias
enhance.fusion.conv.1.running_mean
enhance.fusion.conv.1.running_var
enhance.fusion.conv.1.num_batches_tracked
enhance.conv.0.weight
enhance.conv.0.bias
enhance.conv.1.weight
enhance.conv.1.bias
enhance.conv.1.running_mean
enhance.conv.1.running_var
enhance.conv.1.num_batches_tracked
enhance.blocks.0.0.weight
enhance.blocks.0.0.bias
enhance.blocks.0.1.weight
enhance.blocks.0.1.bias
enhance.blocks.0.1.running_mean
enhance.blocks.0.1.running_var
enhance.blocks.0.1.num_batches_tracked
enhance.out_conv.0.weight
enhance.out_conv.0.bias
calibrate.in_conv.0.weight
calibrate.in_conv.0.bias
calibrate.in_conv.1.weight
calibrate.in_conv.1.bias
calibrate.in_conv.1.running_mean
calibrate.in_conv.1.running_var
calibrate.in_conv.1.num_batches_tracked
calibrate.convs.0.weight
calibrate.convs.0.bias
calibrate.convs.1.weight
calibrate.convs.1.bia

In [14]:
model_dict["enhance.in_conv.0.weight"], model_dict["enhance.fusion.conv.1.weight"]

(tensor([[[[-0.0304, -0.0355,  0.1075],
           [-0.0085,  0.0898,  0.0804],
           [-0.1262,  0.0687, -0.1742]],
 
          [[-0.0835,  0.1682,  0.1076],
           [-0.0210,  0.0949,  0.1862],
           [ 0.1011, -0.0882,  0.0668]],
 
          [[ 0.0273,  0.1336, -0.0521],
           [ 0.0319, -0.0811,  0.1521],
           [ 0.1813, -0.1443, -0.0491]]],
 
 
         [[[-0.1253,  0.1769,  0.0344],
           [-0.0332,  0.1822, -0.1248],
           [-0.1203, -0.1383,  0.0830]],
 
          [[ 0.0855, -0.1518,  0.0277],
           [-0.1201,  0.0537, -0.0077],
           [-0.1427,  0.0816,  0.0941]],
 
          [[ 0.1259, -0.0746, -0.1254],
           [-0.0010, -0.1049,  0.1600],
           [ 0.0322,  0.1669, -0.0909]]],
 
 
         [[[ 0.0769,  0.1791, -0.0385],
           [-0.1011, -0.1475, -0.1761],
           [-0.1107,  0.1097,  0.1614]],
 
          [[ 0.0096, -0.0736,  0.1115],
           [-0.0392,  0.1917, -0.1855],
           [ 0.1371, -0.0118, -0.1051]],
 
          

In [15]:
pretrained_dict = torch.load('./weights/difficult.pt')
pretrained_dict

OrderedDict([('enhance.in_conv.0.weight',
              tensor([[[[ 0.0423, -0.0069,  0.0136],
                        [-0.0192,  0.0466,  0.0282],
                        [ 0.0156, -0.0182,  0.0169]],
              
                       [[ 0.0339,  0.0111,  0.0144],
                        [-0.0120,  0.0499,  0.0189],
                        [-0.0086,  0.0170,  0.0470]],
              
                       [[-0.0239, -0.0022,  0.0248],
                        [ 0.0250,  0.0276,  0.0348],
                        [ 0.0484,  0.0104, -0.0042]]],
              
              
                      [[[ 0.0057,  0.0049,  0.0344],
                        [-0.0021,  0.0367, -0.0550],
                        [-0.0192, -0.0347, -0.0028]],
              
                       [[ 0.0076, -0.0059,  0.0199],
                        [ 0.0053,  0.0772, -0.0614],
                        [-0.0099, -0.0164, -0.0150]],
              
                       [[-0.0284, -0.0597, -0.0340],
              

In [18]:
pretrained_dict["enhance.in_conv.0.weight"]

tensor([[[[ 0.0423, -0.0069,  0.0136],
          [-0.0192,  0.0466,  0.0282],
          [ 0.0156, -0.0182,  0.0169]],

         [[ 0.0339,  0.0111,  0.0144],
          [-0.0120,  0.0499,  0.0189],
          [-0.0086,  0.0170,  0.0470]],

         [[-0.0239, -0.0022,  0.0248],
          [ 0.0250,  0.0276,  0.0348],
          [ 0.0484,  0.0104, -0.0042]]],


        [[[ 0.0057,  0.0049,  0.0344],
          [-0.0021,  0.0367, -0.0550],
          [-0.0192, -0.0347, -0.0028]],

         [[ 0.0076, -0.0059,  0.0199],
          [ 0.0053,  0.0772, -0.0614],
          [-0.0099, -0.0164, -0.0150]],

         [[-0.0284, -0.0597, -0.0340],
          [ 0.0207,  0.0970,  0.0122],
          [ 0.0180,  0.0533,  0.0217]]],


        [[[ 0.0133, -0.0139,  0.0330],
          [-0.0166, -0.0626, -0.0377],
          [ 0.0460, -0.0198,  0.0096]],

         [[ 0.0223,  0.0084,  0.0153],
          [-0.0272, -0.1054, -0.0160],
          [ 0.0330,  0.0107,  0.0054]],

         [[ 0.0294,  0.0017, -0.0296],
     

In [16]:
model_dict.update(pretrained_dict)
# model.load_state_dict(model_dict)
model_dict

OrderedDict([('enhance.in_conv.0.weight',
              tensor([[[[ 0.0423, -0.0069,  0.0136],
                        [-0.0192,  0.0466,  0.0282],
                        [ 0.0156, -0.0182,  0.0169]],
              
                       [[ 0.0339,  0.0111,  0.0144],
                        [-0.0120,  0.0499,  0.0189],
                        [-0.0086,  0.0170,  0.0470]],
              
                       [[-0.0239, -0.0022,  0.0248],
                        [ 0.0250,  0.0276,  0.0348],
                        [ 0.0484,  0.0104, -0.0042]]],
              
              
                      [[[ 0.0057,  0.0049,  0.0344],
                        [-0.0021,  0.0367, -0.0550],
                        [-0.0192, -0.0347, -0.0028]],
              
                       [[ 0.0076, -0.0059,  0.0199],
                        [ 0.0053,  0.0772, -0.0614],
                        [-0.0099, -0.0164, -0.0150]],
              
                       [[-0.0284, -0.0597, -0.0340],
              

In [17]:
model_dict["enhance.in_conv.0.weight"], model_dict["enhance.fusion.conv.1.weight"]

(tensor([[[[ 0.0423, -0.0069,  0.0136],
           [-0.0192,  0.0466,  0.0282],
           [ 0.0156, -0.0182,  0.0169]],
 
          [[ 0.0339,  0.0111,  0.0144],
           [-0.0120,  0.0499,  0.0189],
           [-0.0086,  0.0170,  0.0470]],
 
          [[-0.0239, -0.0022,  0.0248],
           [ 0.0250,  0.0276,  0.0348],
           [ 0.0484,  0.0104, -0.0042]]],
 
 
         [[[ 0.0057,  0.0049,  0.0344],
           [-0.0021,  0.0367, -0.0550],
           [-0.0192, -0.0347, -0.0028]],
 
          [[ 0.0076, -0.0059,  0.0199],
           [ 0.0053,  0.0772, -0.0614],
           [-0.0099, -0.0164, -0.0150]],
 
          [[-0.0284, -0.0597, -0.0340],
           [ 0.0207,  0.0970,  0.0122],
           [ 0.0180,  0.0533,  0.0217]]],
 
 
         [[[ 0.0133, -0.0139,  0.0330],
           [-0.0166, -0.0626, -0.0377],
           [ 0.0460, -0.0198,  0.0096]],
 
          [[ 0.0223,  0.0084,  0.0153],
           [-0.0272, -0.1054, -0.0160],
           [ 0.0330,  0.0107,  0.0054]],
 
          

In [19]:
model = Network(stage=3)

In [6]:
base_weights = torch.load('./weights/difficult.pt')
pretrained_dict = base_weights
for k, v in pretrained_dict.items():
    print(k)
# model_dict = self.state_dict()
# pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
# model_dict.update(pretrained_dict)
# self.load_state_dict(model_dict)

enhance.in_conv.0.weight
enhance.in_conv.0.bias
enhance.conv.0.weight
enhance.conv.0.bias
enhance.conv.1.weight
enhance.conv.1.bias
enhance.conv.1.running_mean
enhance.conv.1.running_var
enhance.conv.1.num_batches_tracked
enhance.blocks.0.0.weight
enhance.blocks.0.0.bias
enhance.blocks.0.1.weight
enhance.blocks.0.1.bias
enhance.blocks.0.1.running_mean
enhance.blocks.0.1.running_var
enhance.blocks.0.1.num_batches_tracked
enhance.out_conv.0.weight
enhance.out_conv.0.bias
calibrate.in_conv.0.weight
calibrate.in_conv.0.bias
calibrate.in_conv.1.weight
calibrate.in_conv.1.bias
calibrate.in_conv.1.running_mean
calibrate.in_conv.1.running_var
calibrate.in_conv.1.num_batches_tracked
calibrate.convs.0.weight
calibrate.convs.0.bias
calibrate.convs.1.weight
calibrate.convs.1.bias
calibrate.convs.1.running_mean
calibrate.convs.1.running_var
calibrate.convs.1.num_batches_tracked
calibrate.convs.3.weight
calibrate.convs.3.bias
calibrate.convs.4.weight
calibrate.convs.4.bias
calibrate.convs.4.running_

In [8]:
sum = 0 
for k, v in pretrained_dict.items():
    if k in model_dict:
        sum+=1
sum

83

In [9]:
len(pretrained_dict)

83